# Restricted Boltzmann Machines

# Part 1 - Data Preprocessing

In [1]:
import pandas as pd
import numpy as np
import torch
from torch.autograd import Variable
import torch.utils.data
import torch.optim as optim
import torch.nn.parallel
import torch.nn as nn

Importing the Training and Test Datasets

In [2]:
test_set = pd.read_csv('/test_set.csv')
train_set = pd.read_csv('/training_set.csv')

train_set
#column 0 = user ID, column 1 = movie ID, column 2 = rating, column 3 = time stamp

,User,Movie,Rating,Timestamp
0,1,661,3,978302109
1,1,914,3,978301968
2,1,3408,4,978300275
3,1,2355,5,978824291
4,1,1287,5,978302039
...,...,...,...,...
750116,6040,1091,1,956716541
750117,6040,1094,5,956704887
750118,6040,562,5,956704746
750119,6040,1096,4,956715648


In [3]:
test_set
#so 250088 + 750121 = 1 million total ratings 

,User,Movie,Rating,Timestamp
0,1,1193,5,978300760
1,1,1197,3,978302268
2,1,2804,5,978300719
3,1,595,5,978824268
4,1,938,4,978301752
...,...,...,...,...
250083,6040,3735,4,960971654
250084,6040,2791,4,956715569
250085,6040,527,5,956704219
250086,6040,2003,1,956716294


Converting our Dataframes into Numpy Arrays

In [4]:
train_set = np.array(train_set, dtype = 'int') #specified the data type for our data, which is all integers anyway since its ratings and IDs
test_set = np.array(test_set, dtype = 'int') #specified the data type for our data, which is all integers anyway since its ratings and IDs

train_set

array([[        1,       661,         3, 978302109],
       [        1,       914,         3, 978301968],
       [        1,      3408,         4, 978300275],
       ...,
       [     6040,       562,         5, 956704746],
       [     6040,      1096,         4, 956715648],
       [     6040,      1097,         4, 956715569]])

Getting the total number of users and movies

In [5]:
num_users = int(max(max(test_set[:,0]),max(train_set[:,0]))) #getting the maximum user ID in the training and test set from all the rows, and column 0 which is the user ID
num_movies = int(max(max(test_set[:,1]),max(train_set[:,1]))) #getting the maximum movie ID in the training and test set from all the rows, and column 1 which is the movie ID

print(num_users) #so max userID is 6040 and max movieID is 3952
print(num_movies)

6040
3952


Converting Datasets into Array with 6040 rows (users) and 3952 columns (ratings)

In [6]:
#Here we're going to create a list of lists. Basically we'll have 6040 lists (# of users), and each list will have 3952 movies (with movies they haven't rated equal to 0). We do this with a numpy array to make it easier to work with pytorch afterwards
def ratingslist(data):
  ratings_list = []
  for user_id in range(1,num_users+1):
    #starting from 1 since first user_ID is 1 and ending on 6040+1 since python doesn't include the upper bound
    movie_id = data[:,1][data[:,0] == user_id] #so here we're taking all the rows from column 1, which is the movie ids from the training/test set. We're also making it so that it only takes the movies watched by a specific user id. To do this, we basically took column 0 and said it must be equal to user_id from the training/test set
    rating_id = data[:,2][data[:,0] == user_id] #so here we're taking all the rows from column 2, which is the ratings from the training/test sets. We're also making it so that it only takes the ratings by a specific user id. To do this, we basically took column 0 and said it must be equal to user_id from the training/test set
    rating = np.zeros(num_movies) #creating a list of 3952 movies initialized with all zeros, so we can then populate it with ratings from users afterwards, and any movies they didn't rate will be given a value of 0
    rating[movie_id - 1] = rating_id #did -1 because python index starts at 0, but movie_id starts with 1
    ratings_list.append(list(rating)) #now adding the ratings of user_id to the ratings list
  return ratings_list

train_set = ratingslist(train_set) #now we're using our function to convert our training set into a list of lists
test_set = ratingslist(test_set) #same above but for test set

In [7]:
print("Number of Rows (users) =",len(test_set)) #so it's 6040 users
print("Number of Cols (ratings) =", len(test_set[0])) #and the number of columns is 3952 ratings (1 for each movie, 0 if unwatched movie)
print("User 1, Movie 661, Rating =",train_set[0][660]) #we can see for user 1, movie 660, the rating given was 3 which reflects the first row in the training set when first imported
print("User 6040, Movie 535, Rating =",test_set[6039][534]) #we can see for user 6040, movie 535, the rating given was 4 which reflects the last row in the test set when first imported

Number of Rows (users) = 6040
Number of Cols (ratings) = 3952
User 1, Movie 661, Rating = 3.0
User 6040, Movie 535, Rating = 4.0


Converting Array into Torch Tensors

In [8]:
#so we could create a neural network with numpy arrays, but pytorch arrays are far more efficient, which is why we're using it
train_set = torch.FloatTensor(train_set) #FloatTensor expects a list of lists
test_set = torch.FloatTensor(test_set) #FloatTensor expects a list of lists

In [9]:
#Converting to Torch Tensor kept the shapes the same
print("Number of Rows (users) =",len(test_set)) #so it's 6040 users
print("Number of Cols (ratings) =", len(test_set[0])) #and the number of columns is 3952 ratings (1 for each movie, 0 if unwatched movie)
print("User 1, Movie 661, Rating =",train_set[0][660]) #we can see for user 1, movie 660, the rating given was 3 which reflects the first row in the training set when first imported
print("User 6040, Movie 535, Rating =",test_set[6039][534]) #we can see for user 6040, movie 535, the rating given was 4 which reflects the last row in the test set when first imported

Number of Rows (users) = 6040
Number of Cols (ratings) = 3952
User 1, Movie 661, Rating = tensor(3.)
User 6040, Movie 535, Rating = tensor(4.)


Converting the ratings into Binary Ratings 1 (Liked) or 0 (Disliked)

In [10]:
train_set[train_set == 0] = -1 #replacing all movies that were rated as 0 (not rated) is now -1
train_set[train_set == 1] = 0 #replacing all 1 or 2 rated movies as disliked
train_set[train_set == 2] = 0 #replacing all 1 or 2 rated movies as disliked
#can't just do <= 2 because then the -1 we converted for 0 ratings would be converted back to 0
train_set[train_set > 2] = 1 #replacing all 3, 4, or 5 rated movies as liked

test_set[test_set == 0] = -1 #replacing all movies that were rated as 0 (not rated) is now 0
test_set[test_set == 1] = 0 #replacing all 1 or 2 rated movies as disliked
test_set[test_set == 2] = 0 #replacing all 1 or 2 rated movies as disliked 
#can't just do <= 2 because then the -1 we converted for 0 ratings would be converted back to 0
test_set[test_set > 2] = 1 #replacing all 3, 4, or 5 rated movies as liked

In [11]:
print("User 117, Movie 343, Rating =",train_set[116][342]) #we can see for user 117, movie 343, no rating was given
print("User 1, Movie 661, Rating =",train_set[0][660]) #we can see for user 1, movie 660, the rating given was 3 which means it was considered a liked film
print("User 6040, Movie 2003, Rating =",test_set[6039][2002]) #we can see for user 6040, movie 2002, the rating given was 0 which means it was considered a disliked film

User 117, Movie 343, Rating = tensor(-1.)
User 1, Movie 661, Rating = tensor(1.)
User 6040, Movie 2003, Rating = tensor(0.)


# Part 2 - RBM Model

Creating the Architecture of the Neural Network - Restricted Boltzmann Machine

In [12]:
class RBM():
  #num_vis is number of visible nodes, num_hid is number of hidden nodes
  def __init__(self, num_vis, num_hid):
    self.W = torch.randn(num_hid, num_vis) #W is the weights of the parameters of the probabilities of the visible nodes given the hidden nodes (prob_vgh). Since we're using PyTorch, this is gonna be a torched tensor
    #So above, we are randomly initializing the weights of a tensor of size num_hid and num_vis based on normal distribution
    #so this initialized the probability, P, of the visibile nodes given the hidden nodes
    self.a = torch.randn(1, num_hid) #Now we are initializing the bias for the probability of the Hidden Nodes given the Visible Nodes, prob_hgv.
    #So above, the bias needs to be 2 dimensions. So the first dimension corresponds to the batch = 1, whereas the second corresponds to the number of hidden nodes. This is because PyTorch cannot accept a single vector of one dimension as an argument, so we create a fake dimension with the batch size
    self.b = torch.randn(1, num_vis) #Now we are initializing the bias for the probabilitiy of the Visible Nodes given the Hidden Nodes, prob_vgh.

  #so the function below returns a sample of different hidden nodes in our RBM according to a certain probability (that will be computed in this same function, prob_hgv). For each of these hidden nodes, this probability is prob_hgv, which is equal to the sigmoid activation function for the hidden nodes
  #x will correspond to the visible neurons, V, in the probability, prob_hgv. The sigmoid activation function is applied to wx, which is the product of w (vector of weights) times x (vector of visible neurons) plus the bias, a (bias for probability of hidden nodes given visible, prob_hgv)
  def sample_h(self, x):
    wx = torch.mm(x, self.W.t()) #.mm is used to create a product of two torch tensors(matrix 1 times matrix 2). self.W.t() is the weights, but transposed, so we can properly multiply the matrices
    activation = wx + self.a.expand_as(wx) #recall that each input vector will not be treat individually, but inside batches. So when we apply the bias, a, we want to make sure its applied to each line of each dimension (so batch size = 1 and num_hid). So the function expand_as makes sure our matrix expands to fit the target matrix, wx, so we can still do the add operation (and therefore the bias applies to every line of each dimension)
    prob_hgv = torch.sigmoid(activation) #This is basically the probabilty that the Hidden Node Activates given the Visible Node (input) i.e if someone likes a drama film, the visible node for said drama film would activate, thus making a hidden node related to drama genre very likely to activate as well
    return prob_hgv, torch.bernoulli(prob_hgv) #so say we have 100 hidden nodes. prob_hgv will return the probability of the i-th hidden node activating given the input. Then using Bernoulli sampling, we take the probabilty of hidden node i (say 70%), and if a random number between 0 and 1 fall below probability_i (<70%), the hidden node activates, and if it's above probability_i (>70%), the hidden node doesn't activate. And we do that for each of our 100 hidden nodes, which is how Bernoulli Sampling works. This results in a vector of 0s and 1s, where 0s correspond to hidden nodes activated by this Bernoulli sampling, and 1 vise versa. This Bernoulli function returns this sampling of hidden nodes to activate.

  #so the function below returns a sample of different visible nodes in our RBM according to a certain probability (that will be computed in this same function, prob_vgh). For each of these visible nodes, this probability is prob_vgh, which is equal to the sigmoid activation function for the visible nodes
  #y will correspond to the hidden neurons, H, in the probability, prob_vgh. The sigmoid activation function is applied to wy, which is the product of w (vector of weights) times y (vector of hidden neurons) plus the bias, b (bias for probability of visible nodes given hidden, prob_vgh)
  def sample_v(self, y):
    wy = torch.mm(y, self.W) #.mm is used to create a product of two torch tensors(matrix 1 times matrix 2). self.W is the weights for prob_vgh, but not transposed since the default shape corresponds to y for matrix multiplication
    activation = wy + self.b.expand_as(wy) #recall that each input vector will not be treat individually, but inside batches. So when we apply the bias, b, we want to make sure its applied to each line of each dimension (so batch size = 1 and num_vis). So the function expand_as makes sure our matrix expands to fit the target matrix, wx, so we can still do the add operation (and therefore the bias applies to every line of each dimension)
    prob_vgh = torch.sigmoid(activation) #This is basically the probabilty that the Visible Node Activates given the Hidden Node (input) i.e if someone likes a dramas, the hidden node for said drama genres would activate, thus making a visible node providing a drama film very likely to activate as well
    return prob_vgh, torch.bernoulli(prob_vgh) #so say we have 6040 visible nodes (num_movies). prob_vgh will return the probability of the i-th visible node activating given the hidden node. Then using Bernoulli sampling, we take the probabilty of visible node i (say 70%), and if a random number between 0 and 1 fall below probability_i (<70%), the visible node activates, and if it's above probability_i (>70%), the hidden node doesn't activate. And we do that for each of our 6040 visible nodes, which is how Bernoulli Sampling works. This results in a vector of 0s and 1s, where 0s correspond to visible nodes activated by this Bernoulli sampling, and 1 vise versa. This Bernoulli function returns this sampling of visible nodes to activate.

  #v0 is the input vector of all the ratings by one user (visible nodes), vk is the visible nodes after k-samplings (k round trips from visible node to hidden node back to visible node) in Contrastive Divergence, ph0 is the vector of probabilities that at the first iteration the hidden nodes activating given the values of v0 (input vector), phk is the probabilty of the hidden nodes activating after K sampling given the value of the visible nodes after k-samplings (vk)
  #The function below will compute the K-Contrastive Divergence algorithm by updating weights and the biases, as we discussed in the theory
  #http://cms.dm.uba.ar/academico/materias/1ercuat2018/probabilidades_y_estadistica_C/5a89b5075af5cbef5becaf419457cdd77cc9.pdf 
  #see page 15 for the algorithm
  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t() #This equation is from the RBM paper. This is the update the weights. The final .t() is to transpose the Weight matrix
    self.b += torch.sum((v0 - vk), 0) #v0 - vk is the actual equation, 0 is just to keep it in a 2 dimensional format. Matrix size of b should be 100 x 3942
    self.a += torch.sum((ph0 - phk), 0) #same as above. Matrix size of should be 100 x 100
    

Calling our class

In [13]:
num_vis = len(train_set[0]) #number of visibile nodes is the number of movies
num_vis

3952

In [14]:
num_hid = 100 #just picking 100 to start with since the optimal number of hidden nodes is hard to predict, but can be tuned later 
batch_size = 100 #just picking 100 to start with, but can be tuned later

In [15]:
rbm = RBM(num_vis, num_hid) #Calling the class we made above
rbm

# Part 3 -Training the RMB Model

In [16]:
import time #Gonna try to keep track of how long it takes to train our model

start_time = time.time()

num_epoch = 20

#http://cms.dm.uba.ar/academico/materias/1ercuat2018/probabilidades_y_estadistica_C/5a89b5075af5cbef5becaf419457cdd77cc9.pdf
for epoch in range(1, num_epoch + 1):
  training_loss = 0 #initializing the training loss
  s = 0. #s is the counter we will use to normalize the training loss by dividing train loss by s
  #For below, since the batch size is 100, means that the first user is 0, and the last user will be number of users (6040) minus the batch size (100), which is 5940. The third argument is the step size, which is the batch size
  for id_user in range(0, num_users - batch_size, batch_size):
    vk = train_set[id_user:id_user+batch_size] #this gives us our batch of 100 users, since it's say id_user=0, then up to id_user=0 + batch size = 100
    v0 = train_set[id_user:id_user+batch_size] #original ratings of movies by the 100 users in the batch size
    ph0,_ = rbm.sample_h(v0) #recall ph0 is the probability that the hidden nodes activate (equal 1) given the visible nodes at the very beginning. The ,_ means we only want to return the first element from our sample_h function, which is prob_hgv
    for k in range(15):
      #So here, we want to continuously update the visible node. To do this, we take the bernoulli sampling element from the sample_h function and use it to get the samples of hidden nodes, which will lead us to get the next visible node values, vk
      _,hk = rbm.sample_h(vk) #hk is the hidden nodes obtained at the k-th step of Contrastive Divergence. The _, means the function only returns the second element, which is the bernoulli sampling. Then we use vk (which is equal to v0 initially) in the iterations to get the probability of hidden nodes given visibile, Ph_given_v
      _,vk = rbm.sample_v(hk) #so here we are updating our vk based on the hk we found above, basically getting bernoulli sample of prob_vgh
      vk[v0 < 0] = v0[v0 < 0] #This is to just make sure that the movies that weren't rated (which equals -1) in the original visible nodes, v0, stay as unrated in the later visible nodes, vk. That way, we dont do training on unrated movies
    phk,_ = rbm.sample_h(vk) #Just getting the final probability of hidden nodes given the final visible nodes after k samples

    rbm.train(v0, vk, ph0, phk) #training it
    training_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0])) #the loss is just the mean difference between the original visible node ratings and the final visible node ratings after k-iterations/samples
    s += 1. #this is just incrementing the counter
  print('epoch: ' + str(epoch) + ' loss: ' + str(training_loss/s))

print(f'Training took {(time.time() - start_time)/60} minutes')
#so on our training set, our model predicts the whether the user likes or dislikes a movie 78% of the time

epoch: 1 loss: tensor(0.2411)
epoch: 2 loss: tensor(0.2287)
epoch: 3 loss: tensor(0.2282)
epoch: 4 loss: tensor(0.2283)
epoch: 5 loss: tensor(0.2285)
epoch: 6 loss: tensor(0.2283)
epoch: 7 loss: tensor(0.2281)
epoch: 8 loss: tensor(0.2280)
epoch: 9 loss: tensor(0.2283)
epoch: 10 loss: tensor(0.2281)
epoch: 11 loss: tensor(0.2276)
epoch: 12 loss: tensor(0.2285)
epoch: 13 loss: tensor(0.2278)
epoch: 14 loss: tensor(0.2284)
epoch: 15 loss: tensor(0.2281)
epoch: 16 loss: tensor(0.2283)
epoch: 17 loss: tensor(0.2280)
epoch: 18 loss: tensor(0.2281)
epoch: 19 loss: tensor(0.2280)
epoch: 20 loss: tensor(0.2284)
Training took 7.624994619687398 minutes


# Part 4 - Testing the RBM Model

In [20]:
test_loss = 0 #initializing the test loss
s = 0. #s is the counter we will use to normalize the test loss by dividing test loss by s

#So for the for loop below, our range is num_users because we're just going to be making a prediction for every user during our test
for id_user in range(num_users):
  v = train_set[id_user:id_user+1] #so here v keeps the training set, because the training set is the input that will be used to activate the hidden neurons for our test set predictions. Because the training set does not have any of the ratings from the test set, we have to use the training set as the input so it can try to predict the ratings of answers it does not contain
  vt = test_set[id_user:id_user+1] #vt is the target visible nodes for the final predictions
  #ph0 was removed because we only needed it for training, not predicting the test set
  
  #so below, we only need k as 1 step (1 round trip of gibbs sampling, so visibile -> hidden -> visibile) because we already trained it to get to the answer in 15 steps before. So now that it's trained already, it should be able to predict in a single step
  #if statement is just checking to make sure that the target visible nodes has movies with ratings, so it's not just empty
  if len(vt[vt >= 0]) > 0:
    #So here, we want to sample the visible node (predict the output). To do this, we take the bernoulli sampling element from the sample_h function and use it to get the samples of hidden nodes, which will lead us to get the next visible node values, v
    _,h = rbm.sample_h(v) 
    _,v = rbm.sample_v(h) #so here we are updating our v based on the h we found above, basically getting bernoulli sample of Pv_given_H
    test_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0])) #the loss is just the mean difference between the original visible node ratings and the final visible node ratings after computing against the hidden nodes
    s += 1. #this is just incrementing the counter
print('test loss = ' + str(test_loss/s))

#Similarly, our model was able to predict whether the use liked or disliked new movies 80% of the time

test loss = tensor(0.2043)
